### 1. 라이브러리 불러오기

In [6]:
import warnings
warnings.filterwarnings(action="ignore")

In [7]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow import keras
%matplotlib inline

### 2. 이미지 전처리 함수 생성

In [19]:
def refineImage(testImg):
    ## 테스트 이미지 컬러 변경
    ## 컬러이미지 -> 회색톤으로 변경
    IMG_SIZE = 28
    img_gray= cv2.cvtColor(testImg,cv2.COLOR_RGB2GRAY)

    ## 사이즈 조정
    img_resize = cv2.resize(img_gray,(IMG_SIZE,IMG_SIZE))

    ## shape 변경
    img_shape = img_resize.reshape(1,IMG_SIZE,IMG_SIZE,1)

    img_shape.shape

    ## 정규화
    img_norm = img_shape/255.0
    
    return img_norm

### 3. 모델 불러오기

In [9]:
from tensorflow.keras.models import model_from_json

In [11]:
json_file = open("d:/model.json","r")
loaded_model_json = json_file.read()
json_file.close()

In [14]:
loaded_model = model_from_json(loaded_model_json)

In [15]:
loaded_model.load_weights("d:\model_mnist_kopo_weight.h5")

In [16]:
testImg = cv2.imread("d:\sample_image.PNG",cv2.IMREAD_COLOR)

In [20]:
outimg = refineImage(testImg)

In [21]:
np.argmax(loaded_model.predict(outimg))

5

In [22]:
outimg.shape

(1, 28, 28, 1)

### 4. 이미지 캡쳐(비디오 캡쳐)

In [37]:
cap = cv2.VideoCapture(0)
width = int(cap.get (cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get (cv2.CAP_PROP_FRAME_HEIGHT))
CAP_SIZE = 150
while(True):
    ret,frame = cap.read()
    if ret == False:
        break;
    
    x1 = int(width  /2 - CAP_SIZE/2)
    y1 = int(height /2 - CAP_SIZE/2)
    x2 = x1 + CAP_SIZE
    y2 = y1 + CAP_SIZE
    
#     cv2.imshow('color_frame', frame)
    key = cv2.waitKey(33)
    frame2 = cv2.rectangle(frame, (x1,y1), (x2,y2),(0,0,255),2)
    rectImage = frame2[y1:y1 + CAP_SIZE, x1:x1 + CAP_SIZE]
    outframe = refineImage(rectImage)
    predictValue = np.argmax(loaded_model.predict(outframe))
    
    cv2.putText(frame2,
               str(predictValue),
                (x1,y1-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0,255,255)
               )
    
    cv2.imshow('color_frame', frame2)
    
    if key==27:
        # esc key
        cap.release()
        cv2.destroyAllWindows()
    
    elif key==32:
        rectImage = frame2[y1:y1 + CAP_SIZE, x1:x1 + CAP_SIZE]
        # space bar key
        outframe = refineImage(rectImage)
        predictValue = np.argmax(loaded_model.predict(outframe))
        print(predictValue)
#         cv2.imwrite("d:/capture_frame_images.jpg", frame )
#         cv2.imwrite("d:/capture_frame2_images.jpg", frame2 )
        cv2.imwrite("d:/capture_rec_images.jpg", rectImage )
        
cap.release()
cv2.destroyAllWindows()

0
1
8


### 5. 영역 설정

### 6. 예측